# Multi-layer LSTM for Music Generation (with Magenta)

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout, Activation

In [4]:
def create_network(network_input, n_vocab):
    """ Define constants """
    hidden_layers = 256
    dropout = 0.4
    temp = 0.6
    
    """ Initializing model """
    model = Sequential()
    
    """ Adding LSTM Layers to Model """
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True,
                input_shape=(network_input.shape[1], network_input.shape[2])
            )
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            )
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout
            )
        )
    )
    
    """ Add other layers after LSTM """
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layers // 2))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda x: x / temp))
    model.add(Softmax())
    
    """ Define the optimizer and loss function for the model """
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    return model
    

In [3]:
import os
from magenta.scripts.convert_dir_to_note_sequences import convert_directory
from note_seq import music_pb2

testing_file = 'testing_sequences'
training_file = 'training_sequences'
valid_file = 'valid_sequences'

# convert_directory(os.path.join(os.getcwd(), 'nesmdb_midi', 'test'), testing_file, True)
# convert_directory(os.path.join(os.getcwd(), 'nesmdb_midi', 'train'), training_file, True)
# convert_directory(os.path.join(os.getcwd(), 'nesmdb_midi', 'valid'), valid_file, True)

In [44]:
reader = tf.data.TFRecordDataset(testing_file)
sequences = []
for sequence in reader:
    # sequences.append(music_pb2.NoteSequence.FromString(sequence))
    sequences.append(sequence)
data = music_pb2.NoteSequence.FromString(sequences[0].numpy()).notes
data.sort(key=lambda e: e.start_time)
data
music_pb2.NoteSequence.FromString(sequences[1].numpy()).instrument_infos
#""".sort(key=lambda i: i.)"""
# data = list(data).sort(key=sortStart)
#for e in data:
#    print(e.start_time)

[name: "p1"
, instrument: 1
name: "p2"
, instrument: 2
name: "no"
]